In [1]:
pip install open-clip-torch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 88.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 131.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 99.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 64.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 41.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 19.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 106.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.8/44.8 kB 4.8 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu1

In [2]:
# ╔══════════════════════════════════════════════════════════════════════╗
# ║  Magma-8B Multi-Technique Demo with PDF Report Generation            ║
# ║  (works on Colab Pro, tested with A100)                              ║
# ╚══════════════════════════════════════════════════════════════════════╝

# 1️⃣  Install deps (first run only) ------------------------------------------
# (comment these out after the runtime already has the wheels cached)
!pip install -q --upgrade git+https://github.com/jwyang/transformers.git@dev/jwyang-v4.48.2 \
                      torchvision Pillow open_clip_torch reportlab

# 2️⃣  Imports & model ---------------------------------------------------------
import io, requests, torch, typing as T, os
from collections import Counter
from datetime import datetime
from PIL import Image
from google.colab import files
from transformers import AutoModelForCausalLM, AutoProcessor
from reportlab.lib.pagesizes import letter, A4
from reportlab.platypus import SimpleDocTemplate, Paragraph, Spacer, Image as RLImage, PageBreak
from reportlab.lib.styles import getSampleStyleSheet, ParagraphStyle
from reportlab.lib.units import inch
from reportlab.lib.enums import TA_CENTER, TA_LEFT
import tempfile

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
DTYPE  = torch.bfloat16 if DEVICE == "cuda" else torch.float32

print("⏳ Loading Magma-8B …")
MODEL = AutoModelForCausalLM.from_pretrained(
            "microsoft/Magma-8B",
            trust_remote_code=True,
            torch_dtype=DTYPE).to(DEVICE).eval()

PROCESSOR = AutoProcessor.from_pretrained("microsoft/Magma-8B",
                                          trust_remote_code=True)
print("✅  Magma-8B ready")

# 3️⃣  Prompt builder with clear section tags ----------------------------------
def prompt_builder(
    task: str,
    flavour: str = "standard",
    *,
    reasoning_tag: str = "### Reasoning:",
    draft_tag:    str = "### Draft:",
    refined_tag:  str = "### Refined Answer:",
    answer_tag:   str = "### Answer:"
) -> list[dict[str, str]]:
    """Return a list of chat messages for .apply_chat_template()."""
    flavour = flavour.lower()
    if flavour not in {"standard", "cot", "cod", "cot_consistency"}:
        raise ValueError("flavour must be standard / cot / cod / cot_consistency")

    # ---- build the user message body ---------------------------------------
    if flavour == "standard":
        user_body = (
            "<image_start><image><image_end>\n"
            f"{task}\nPlease answer briefly and accurately."
        )
    elif flavour == "cot":
        user_body = (
            "<image_start><image><image_end>\n"
            f"{task}\n{reasoning_tag} Think step-by-step, then end with:\n{answer_tag}"
        )
    elif flavour == "cod":
        user_body = (
            "<image_start><image><image_end>\n"
            f"{task}\n{draft_tag} Bullet thoughts (≤ 6 words each). "
            f"Rewrite a polished reply under:\n{refined_tag}"
        )
    else:  # cot_consistency
        user_body = (
            "<image_start><image><image_end>\n"
            f"{task}\n{reasoning_tag} Think independently. "
            f"Finish with:\n{answer_tag}"
        )

    return [
        {
            "role": "system",
            "content": (
                "You are a vision-language agent that analyses indoor scenes "
                "and plans safe, precise actions for a 170 cm humanoid robot."
            ),
        },
        {"role": "user", "content": user_body},
    ]

# 4️⃣  Inference helper ---------------------------------------------------------
def magma_analyse(
    image: Image.Image,
    task: str,
    flavour: str = "standard",
    *,
    max_new: int = 256,
    num_samples: int = 5,               # only for cot_consistency
    temperature: float = 0.7,
    top_p: float = 0.9,
) -> T.Union[str, tuple[str, list[str]]]:
    """Run Magma-8B with one of four prompting techniques."""
    # ---- build chat prompt --------------------------------------------------
    chat_prompt = PROCESSOR.tokenizer.apply_chat_template(
        prompt_builder(task, flavour),
        tokenize=False,
        add_generation_prompt=True,
    )

    # ---- encode inputs ------------------------------------------------------
    inputs = PROCESSOR(images=[image], texts=chat_prompt, return_tensors="pt")
    inputs["pixel_values"] = inputs["pixel_values"].unsqueeze(0).to(DEVICE).to(DTYPE)
    inputs["image_sizes"]  = inputs["image_sizes"].unsqueeze(0).to(DEVICE)
    inputs["input_ids"]    = inputs["input_ids"].to(DEVICE)
    inputs["attention_mask"] = inputs["attention_mask"].to(DEVICE)

    gen_cfg = dict(
        max_new_tokens=max_new,
        do_sample=True,
        temperature=temperature,
        top_p=top_p,
        use_cache=True,
        repetition_penalty=1.1,
    )

    # ---- single-shot modes --------------------------------------------------
    if flavour in {"standard", "cot", "cod"}:
        with torch.inference_mode():
            ids = MODEL.generate(**inputs, **gen_cfg)
        answer = PROCESSOR.decode(
            ids[0, inputs["input_ids"].shape[-1]:],
            skip_special_tokens=True,
        ).strip()
        return answer

    # ---- self-consistency mode ---------------------------------------------
    answers = []
    with torch.inference_mode():
        for _ in range(num_samples):
            ids = MODEL.generate(**inputs, **gen_cfg)
            ans = PROCESSOR.decode(
                ids[0, inputs["input_ids"].shape[-1]:],
                skip_special_tokens=True,
            ).strip()
            answers.append(ans)

    majority = Counter(answers).most_common(1)[0][0]
    return majority, answers  # return both the vote and raw samples

# 5️⃣  Multi-technique analysis function ------------------------------------
def run_all_techniques(image: Image.Image, task: str) -> dict:
    """Run all four techniques on the same image and task."""
    results = {}
    techniques = ["standard", "cot", "cod", "cot_consistency"]

    print("🚀 Running all prompting techniques...")

    for technique in techniques:
        print(f"⏳ Running {technique.upper()}...")

        if technique == "cot_consistency":
            majority, samples = magma_analyse(image, task, flavour=technique)
            results[technique] = {
                "majority": majority,
                "samples": samples,
                "type": "consistency"
            }
        else:
            answer = magma_analyse(image, task, flavour=technique)
            results[technique] = {
                "answer": answer,
                "type": "single"
            }

        print(f"✅ {technique.upper()} completed")

    return results

# 6️⃣  PDF Report Generator -------------------------------------------------
def create_pdf_report(image: Image.Image, task: str, results: dict, filename: str = None):
    """Generate a comprehensive PDF report with all results."""
    if filename is None:
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        filename = f"magma_8b_analysis_{timestamp}.pdf"

    # Create document
    doc = SimpleDocTemplate(filename, pagesize=A4)
    story = []
    styles = getSampleStyleSheet()

    # Custom styles
    title_style = ParagraphStyle(
        'CustomTitle',
        parent=styles['Heading1'],
        fontSize=18,
        spaceAfter=30,
        alignment=TA_CENTER
    )

    heading_style = ParagraphStyle(
        'CustomHeading',
        parent=styles['Heading2'],
        fontSize=14,
        spaceAfter=12,
        spaceBefore=20
    )

    subheading_style = ParagraphStyle(
        'CustomSubHeading',
        parent=styles['Heading3'],
        fontSize=12,
        spaceAfter=8,
        spaceBefore=10
    )

    # Title
    story.append(Paragraph("Magma-8B Multi-Technique Analysis Report", title_style))
    story.append(Spacer(1, 20))

    # Metadata
    timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    story.append(Paragraph(f"<b>Generated:</b> {timestamp}", styles['Normal']))
    story.append(Paragraph(f"<b>Model:</b> Microsoft Magma-8B", styles['Normal']))
    story.append(Paragraph(f"<b>Task/Prompt:</b> {task}", styles['Normal']))
    story.append(Spacer(1, 20))

    # Save image temporarily for PDF
    with tempfile.NamedTemporaryFile(suffix='.png', delete=False) as tmp_file:
        image.save(tmp_file.name, 'PNG')
        tmp_image_path = tmp_file.name

    # Add image
    story.append(Paragraph("Input Image", heading_style))
    img = RLImage(tmp_image_path, width=4*inch, height=3*inch)
    story.append(img)
    story.append(Spacer(1, 20))

    # Results section
    story.append(Paragraph("Analysis Results", heading_style))
    story.append(Spacer(1, 10))

    # Technique descriptions
    technique_descriptions = {
        "standard": "Direct prompting without additional reasoning steps",
        "cot": "Chain-of-Thought: Step-by-step reasoning before final answer",
        "cod": "Chain-of-Draft: Draft thoughts followed by refined response",
        "cot_consistency": "Self-Consistency: Multiple reasoning paths with majority voting"
    }

    for technique in ["standard", "cot", "cod", "cot_consistency"]:
        story.append(Paragraph(f"{technique.upper()} Technique", subheading_style))
        story.append(Paragraph(f"<i>{technique_descriptions[technique]}</i>", styles['Normal']))
        story.append(Spacer(1, 8))

        if results[technique]["type"] == "single":
            answer = results[technique]["answer"]
            story.append(Paragraph("<b>Response:</b>", styles['Normal']))
            story.append(Paragraph(answer, styles['Normal']))
        else:  # consistency
            majority = results[technique]["majority"]
            samples = results[technique]["samples"]

            story.append(Paragraph("<b>Majority Vote Result:</b>", styles['Normal']))
            story.append(Paragraph(majority, styles['Normal']))
            story.append(Spacer(1, 8))

            story.append(Paragraph("<b>All Samples:</b>", styles['Normal']))
            for i, sample in enumerate(samples, 1):
                story.append(Paragraph(f"Sample {i}: {sample}", styles['Normal']))

        story.append(Spacer(1, 20))

    # Summary section
    story.append(PageBreak())
    story.append(Paragraph("Analysis Summary", heading_style))
    story.append(Paragraph(
        "This report compares four different prompting techniques applied to the Magma-8B vision-language model. "
        "Each technique has different strengths:", styles['Normal']
    ))
    story.append(Spacer(1, 10))

    story.append(Paragraph("• <b>Standard:</b> Fast and direct, good for simple tasks", styles['Normal']))
    story.append(Paragraph("• <b>CoT:</b> Better reasoning for complex tasks requiring step-by-step thinking", styles['Normal']))
    story.append(Paragraph("• <b>CoD:</b> Iterative refinement for improved response quality", styles['Normal']))
    story.append(Paragraph("• <b>CoT-Consistency:</b> Most robust but computationally expensive, reduces hallucinations", styles['Normal']))

    # Build PDF
    doc.build(story)

    # Clean up temporary image
    os.unlink(tmp_image_path)

    print(f"📄 PDF report saved as: {filename}")
    return filename

# 7️⃣  Main interactive function --------------------------------------------
def upload_and_analyze_all():
    """Upload image, get prompt, run all techniques, generate PDF report."""
    print("📁 Please upload an image...")
    uploaded = files.upload()
    img_path = next(iter(uploaded))
    img = Image.open(img_path).convert("RGB")

    print("🖼️  Image loaded successfully!")

    # Get task/prompt from user
    task = input("Enter your prompt/task (e.g., 'go towards a woman'): ").strip()
    if not task:
        task = "go towards a woman"  # default
        print(f"Using default prompt: '{task}'")

    print(f"\n🎯 Task: {task}")
    print("=" * 60)

    # Run all techniques
    results = run_all_techniques(img, task)

    # Display results
    print("\n" + "=" * 60)
    print("📊 RESULTS SUMMARY")
    print("=" * 60)

    for technique in ["standard", "cot", "cod", "cot_consistency"]:
        print(f"\n🔸 {technique.upper()}:")
        print("-" * 40)

        if results[technique]["type"] == "single":
            print(results[technique]["answer"])
        else:  # consistency
            print("Majority Answer:", results[technique]["majority"])
            print(f"\nAll {len(results[technique]['samples'])} samples:")
            for i, sample in enumerate(results[technique]["samples"], 1):
                print(f"  [{i}] {sample}")

    # Generate PDF report
    print("\n" + "=" * 60)
    print("📄 Generating PDF Report...")
    pdf_filename = create_pdf_report(img, task, results)

    # Download the PDF
    print("⬇️  Downloading PDF report...")
    files.download(pdf_filename)

    print("\n✅ Analysis complete! Check your downloads for the PDF report.")

# 8️⃣  Alternative function for batch processing ------------------------
def analyze_with_custom_params(
    image_path: str,
    task: str,
    output_filename: str = None,
    max_new_tokens: int = 256,
    temperature: float = 0.7,
    num_consistency_samples: int = 5
):
    """Analyze with custom parameters - useful for scripted runs."""
    img = Image.open(image_path).convert("RGB")

    # Custom analysis with different parameters
    results = {}
    techniques = ["standard", "cot", "cod", "cot_consistency"]

    for technique in techniques:
        if technique == "cot_consistency":
            majority, samples = magma_analyse(
                img, task, flavour=technique,
                max_new=max_new_tokens, temperature=temperature,
                num_samples=num_consistency_samples
            )
            results[technique] = {"majority": majority, "samples": samples, "type": "consistency"}
        else:
            answer = magma_analyse(
                img, task, flavour=technique,
                max_new=max_new_tokens, temperature=temperature
            )
            results[technique] = {"answer": answer, "type": "single"}

    # Generate PDF
    pdf_filename = create_pdf_report(img, task, results, output_filename)
    return results, pdf_filename

# 👉  Run the enhanced demo
print("\n🎉 Enhanced Magma-8B Multi-Technique Demo Ready!")
print("This will run all 4 techniques automatically and generate a PDF report.")
print("\nStarting analysis...")
upload_and_analyze_all()

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.5/7.5 MB 132.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 821.2/821.2 MB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 393.1/393.1 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.9/8.9 MB 124.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 99.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 897.7/897.7 kB 62.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 571.0/571.0 MB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.2/200.2 MB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 70.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 44.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.2/158.2 MB 16.6 MB/s eta

config.json:   0%|          | 0.00/3.25k [00:00<?, ?B/s]

configuration_magma.py:   0%|          | 0.00/7.60k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/microsoft/Magma-8B:
- configuration_magma.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


modeling_magma.py:   0%|          | 0.00/75.6k [00:00<?, ?B/s]

...

/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:838: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.11/dist-packages/torch/utils/checkpoint.py:86: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(
The `seen_tokens` attribute is deprecated and will be removed in v4.41. Use the `cache_position` model input instead.


✅ STANDARD completed
⏳ Running COT...
✅ COT completed
⏳ Running COD...
✅ COD completed
⏳ Running COT_CONSISTENCY...
✅ COT_CONSISTENCY completed

📊 RESULTS SUMMARY

🔸 STANDARD:
----------------------------------------
You can take the green staircase on the left side of the room.

🔸 COT:
----------------------------------------
To get to the first floor, you would need to walk down the green staircase on the left side of the image. The staircase leads from the second floor to the first floor, providing access to the lower level of the house.

🔸 COD:
----------------------------------------
To reach the first floor, take the green staircase located in the living room.

🔸 COT_CONSISTENCY:
----------------------------------------
Majority Answer: The first floor is accessible via a set of green stairs located in the living room.

All 5 samples:
  [1] The first floor is accessible via a set of green stairs located in the living room.
  [2] To get to the first floor, you can use the green st

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>


✅ Analysis complete! Check your downloads for the PDF report.


In [3]:
upload_and_analyze_all()

📁 Please upload an image...


Saving Kitchen.jpg to Kitchen.jpg
🖼️  Image loaded successfully!
Enter your prompt/task (e.g., 'go towards a woman'): Can you go towards the woman?

🎯 Task: Can you go towards the woman?
🚀 Running all prompting techniques...
⏳ Running STANDARD...
✅ STANDARD completed
⏳ Running COT...
✅ COT completed
⏳ Running COD...
✅ COD completed
⏳ Running COT_CONSISTENCY...
✅ COT_CONSISTENCY completed

📊 RESULTS SUMMARY

🔸 STANDARD:
----------------------------------------
Yes

🔸 COT:
----------------------------------------
Sure! Let's start by observing the scene. The kitchen is spacious and well-lit, with white walls and gray cabinets. A large stainless steel refrigerator stands on the left side of the room, while a black island takes center stage in the middle. Three wicker barstools are tucked under the island, providing seating.

A woman is seated at the island, engrossed in her phone. She’s dressed casually in a white shirt and blue jeans. The island houses a sink and a stove, ready for culin

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>


✅ Analysis complete! Check your downloads for the PDF report.


In [4]:
upload_and_analyze_all()

📁 Please upload an image...


Saving home.jpg to home.jpg
🖼️  Image loaded successfully!
Enter your prompt/task (e.g., 'go towards a woman'): Can you go towards the lamp?

🎯 Task: Can you go towards the lamp?
🚀 Running all prompting techniques...
⏳ Running STANDARD...
✅ STANDARD completed
⏳ Running COT...
✅ COT completed
⏳ Running COD...
✅ COD completed
⏳ Running COT_CONSISTENCY...
✅ COT_CONSISTENCY completed

📊 RESULTS SUMMARY

🔸 STANDARD:
----------------------------------------
Yes

🔸 COT:
----------------------------------------
Yes, I can walk towards the floor lamp.

🔸 COD:
----------------------------------------
Yes, I can walk towards the lamp.

🔸 COT_CONSISTENCY:
----------------------------------------
Majority Answer: Yes, I can walk towards the lamp.

All 5 samples:
  [1] Yes, I can walk towards the lamp.
  [2] Yes, I can walk towards the floor lamp in the living room.
  [3] Yes, I can walk towards the lamp.
  [4] No, I cannot reach the lamp as it is too far away from me. I am only a computer program a

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>


✅ Analysis complete! Check your downloads for the PDF report.


In [5]:
upload_and_analyze_all()



📁 Please upload an image...


Saving kitchen_map.JPG to kitchen_map.JPG
🖼️  Image loaded successfully!
Enter your prompt/task (e.g., 'go towards a woman'): How can you go towards the commercial dishwasher?

🎯 Task: How can you go towards the commercial dishwasher?
🚀 Running all prompting techniques...
⏳ Running STANDARD...
✅ STANDARD completed
⏳ Running COT...
✅ COT completed
⏳ Running COD...
✅ COD completed
⏳ Running COT_CONSISTENCY...
✅ COT_CONSISTENCY completed

📊 RESULTS SUMMARY

🔸 STANDARD:
----------------------------------------
兄弟 〜ｍ衝ẳn بلغุษย

🔸 COT:
----------------------------------------
To reach the commercial dishwasher from the main kitchen, follow these steps:

1. Move towards the right side of the room.
2. Continue moving right until you reach the commercial dishwasher area.

The commercial dishwasher is located on the right side of the room can be accessed by following these steps.

🔸 COD:
----------------------------------------
To reach the commercial dishwasher, follow the path from the kitchen

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>


✅ Analysis complete! Check your downloads for the PDF report.


In [6]:
upload_and_analyze_all()


📁 Please upload an image...


Saving office-retail-category-desks-tables.jpg to office-retail-category-desks-tables.jpg
🖼️  Image loaded successfully!
Enter your prompt/task (e.g., 'go towards a woman'): How can you locate and move towards any chair in the office?

🎯 Task: How can you locate and move towards any chair in the office?
🚀 Running all prompting techniques...
⏳ Running STANDARD...
✅ STANDARD completed
⏳ Running COT...
✅ COT completed
⏳ Running COD...
✅ COD completed
⏳ Running COT_CONSISTENCY...
✅ COT_CONSISTENCY completed

📊 RESULTS SUMMARY

🔸 STANDARD:
----------------------------------------
огля एपکات ослож 手机 домовุษย

🔸 COT:
----------------------------------------
olet Григореком斷 kaliteli 있어서ุษย

🔸 COD:
----------------------------------------
огля एपکات ослож 手机 домовุษย

🔸 COT_CONSISTENCY:
----------------------------------------
Majority Answer: oletанню……。 手机 Peygamberційнаiв

All 5 samples:
  [1] oletанню……。 手机 Peygamberційнаiв
  [2] ือขौडアニメаннюつぶ efektุษย
  [3] olet……。Nghือข منزلفوุษย
  [4]

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>


✅ Analysis complete! Check your downloads for the PDF report.


In [7]:
upload_and_analyze_all()

📁 Please upload an image...


Saving room_map.jpg to room_map.jpg
🖼️  Image loaded successfully!
Enter your prompt/task (e.g., 'go towards a woman'): Bring me a piece of a paper

🎯 Task: Bring me a piece of a paper
🚀 Running all prompting techniques...
⏳ Running STANDARD...
✅ STANDARD completed
⏳ Running COT...
✅ COT completed
⏳ Running COD...
✅ COD completed
⏳ Running COT_CONSISTENCY...
✅ COT_CONSISTENCY completed

📊 RESULTS SUMMARY

🔸 STANDARD:
----------------------------------------
The paper is located on the desk in the room with the bed and round rug.

🔸 COT:
----------------------------------------
the camera wearer takes the paper on the table

🔸 COD:
----------------------------------------
1. Plan actions for robot's tasks.
2. Ensure precision and safety.
3. Indoor scene analysis required.
4. 170cm humanoid robot target.
5. Safe actions for indoors.
6. Precise movements in rooms.
7. Avoid obstacles and hazards.
8. Efficient navigation within space.
9. Adapt to changing environments.
10. Considerate priva

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>


✅ Analysis complete! Check your downloads for the PDF report.


In [8]:
upload_and_analyze_all()

📁 Please upload an image...


Saving office_layout.jpg to office_layout.jpg
🖼️  Image loaded successfully!
Enter your prompt/task (e.g., 'go towards a woman'): How can you go to the Eddy table?

🎯 Task: How can you go to the Eddy table?
🚀 Running all prompting techniques...
⏳ Running STANDARD...
✅ STANDARD completed
⏳ Running COT...
✅ COT completed
⏳ Running COD...
✅ COD completed
⏳ Running COT_CONSISTENCY...
✅ COT_CONSISTENCY completed

📊 RESULTS SUMMARY

🔸 STANDARD:
----------------------------------------
oletiв炉つぶยนแปลง私のุษย

🔸 COT:
----------------------------------------
兄弟族自治کاتิวเตอร 드라마ційнаุษย

🔸 COD:
----------------------------------------
To reach the Eddy table, follow these steps:

1. Move towards the kitchen area.
2. Turn right from the hallway.
3. Follow the signs for the Eddy room.

### Original thoughts:
- Go to the kitchen.
- Find the Eddy table.

### Refinement:
- To quickly locate the Eddy table, head towards the kitchen area and turn right at the signs indicating its direction.

🔸 COT_CONSIST

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>


✅ Analysis complete! Check your downloads for the PDF report.


In [9]:
upload_and_analyze_all()

📁 Please upload an image...


Saving office_marked.JPG to office_marked.JPG
🖼️  Image loaded successfully!
Enter your prompt/task (e.g., 'go towards a woman'): From the entrance, go towards the green sofa

🎯 Task: From the entrance, go towards the green sofa
🚀 Running all prompting techniques...
⏳ Running STANDARD...
✅ STANDARD completed
⏳ Running COT...
✅ COT completed
⏳ Running COD...
✅ COD completed
⏳ Running COT_CONSISTENCY...
✅ COT_CONSISTENCY completed

📊 RESULTS SUMMARY

🔸 STANDARD:
----------------------------------------
the camera wearer goes up the stairs

🔸 COT:
----------------------------------------
The green sofa is located in the bottom right corner of the image, near the center of the room. To get there, walk straight ahead from the entrance, passing by the bookshelves on your left and the TV on your right. Continue moving forward until you reach the green sofa.

🔸 COD:
----------------------------------------
1. Please proceed to the green sofa.
2. You will find a comfortable seating area there.


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>


✅ Analysis complete! Check your downloads for the PDF report.


In [10]:
upload_and_analyze_all()

📁 Please upload an image...


Saving office_plan.PNG to office_plan.PNG
🖼️  Image loaded successfully!
Enter your prompt/task (e.g., 'go towards a woman'): Go towards the big oval table 

🎯 Task: Go towards the big oval table
🚀 Running all prompting techniques...
⏳ Running STANDARD...
✅ STANDARD completed
⏳ Running COT...
✅ COT completed
⏳ Running COD...
✅ COD completed
⏳ Running COT_CONSISTENCY...
✅ COT_CONSISTENCY completed

📊 RESULTS SUMMARY

🔸 STANDARD:
----------------------------------------
Coordinate: (0.87, 0.21)

🔸 COT:
----------------------------------------
the camera wearer moves around the room

🔸 COD:
----------------------------------------
1. The dining area is located in the bottom right corner of the floor plan.
2. There are two types of tables available: round and rectangular.
3. The large oval table is situated in the top right corner of the floor plan.
4. The conference room is located in the top left corner of the floor plan.
5. There is a microwave located in the bottom left corner of the f

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>


✅ Analysis complete! Check your downloads for the PDF report.


In [11]:
upload_and_analyze_all()

📁 Please upload an image...


Saving the-ultimate-home-office-design.jpg to the-ultimate-home-office-design.jpg
🖼️  Image loaded successfully!
Enter your prompt/task (e.g., 'go towards a woman'): Where is the laptop located?

🎯 Task: Where is the laptop located?
🚀 Running all prompting techniques...
⏳ Running STANDARD...
✅ STANDARD completed
⏳ Running COT...
✅ COT completed
⏳ Running COD...
✅ COD completed
⏳ Running COT_CONSISTENCY...
✅ COT_CONSISTENCY completed

📊 RESULTS SUMMARY

🔸 STANDARD:
----------------------------------------
Desk

🔸 COT:
----------------------------------------
The laptop is located on top of a wooden bookshelf in a library-like room. This suggests that the room might be used for studying or working, as it has a desk and chair setup nearby. It could also imply that the person who uses this space prefers to have their work area close to their book collection, possibly indicating an interest in both traditional and digital resources. The presence of a calendar on the wall further supports th

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>


✅ Analysis complete! Check your downloads for the PDF report.
